<a href="https://colab.research.google.com/github/lennart194/thesis-code/blob/main/merge_data_m5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from os import rename
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

## Import Data

In [ ]:
calendar = pd.read_csv('/content/drive/MyDrive/M5/calendar.csv')
sales_data = pd.read_csv('/content/drive/MyDrive/M5/sales_train_evaluation.csv')
sell_prices = pd.read_csv('/content/drive/MyDrive/M5/sell_prices.csv')

## Edit sales_data

*   drop all rows where cat_id isnot FOODS
*   drop all rows where store_id isnot CA_3
*   set the index to item_id
*   create a variable day_cols containing all the d_columns



In [ ]:
sales_data = sales_data[sales_data['cat_id'].str.contains('FOODS') == True]
sales_data = sales_data[sales_data['store_id'].str.contains('CA_3') == True]
sales_data = sales_data.set_index('item_id')
day_cols = [col for col in sales_data.columns if 'd_' in col]

### Find the 25 Products with the lowest intermittend demand

*   create a new data-set sales_data_dcols with index item_id containing only the day_cols
*   create a new data-set sales_data_intermittend with index item_id and the sorted count of intermittend demand
*   reset the index and change the column name to intermittend_demand
*   choose the first 25 entries





In [ ]:
sales_data_dcols = sales_data[day_cols]
sales_data_intermittend = sales_data_dcols[sales_data_dcols == 0].count(axis=1).sort_values()
sales_data_intermittend = sales_data_intermittend.reset_index()
sales_data_intermittend = sales_data_intermittend.rename(columns={0:"intermittend_demand"})
sales_data_intermittend = sales_data_intermittend.head(25)

In [ ]:
sales_data_intermittend

,item_id,intermittend_demand
0,FOODS_3_555,4
1,FOODS_3_080,5
2,FOODS_3_586,5
3,FOODS_3_389,5
4,FOODS_3_228,6
5,FOODS_3_252,6
6,FOODS_3_694,6
7,FOODS_3_714,8
8,FOODS_3_377,10
9,FOODS_3_668,10


### Inner merge sales_data and sales_data_intermittend on item_id and drop some columns

In [ ]:
sales_data = sales_data.merge(sales_data_intermittend, how='inner', on='item_id')
sales_data = sales_data.drop(columns=['id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'intermittend_demand'])

## Edit calendar

*   create one single column for event_type
*   create dummys for every event type
*   delete old columns
*   drop columns snap_TX & snap_WI and rename snap_CA

In [ ]:
calendar['event_type'] = calendar['event_type_1'].astype(str) + '&' + calendar['event_type_2'].astype(str)

calendar['is_sporting_event'] = calendar['event_type'].apply (lambda row: 1 if 'Sporting' in row else 0)
calendar['is_national_event'] = calendar['event_type'].apply (lambda row: 1 if 'National' in row else 0)
calendar['is_cultural_event'] = calendar['event_type'].apply (lambda row: 1 if 'Cultural' in row else 0)
calendar['is_religious_event'] = calendar['event_type'].apply (lambda row: 1 if 'Religious' in row else 0)
calendar = calendar.drop(columns=['event_name_1', 'event_name_2', 'event_type_1', 'event_type_2', 'event_type'])
calendar = calendar.drop(columns=['snap_TX', 'snap_WI'])
calendar = calendar.rename(columns={'snap_CA':'is_snap_event'})

*   convert year into object type
*   LabelEncoding of year







In [ ]:
calendar['year'] = calendar['year'].astype(object)

labenc = LabelEncoder()
calendar['year'] = labenc.fit_transform(calendar['year'])

*   OneHotEncoding of weekdays, months (change type to int)
*   delete weekday, wday and month


In [ ]:
ohe = OneHotEncoder()
date_array = ohe.fit_transform(calendar[['weekday', 'month']]).toarray()
date_labels = ohe.categories_
date_labels = np.hstack(date_labels)
date = pd.DataFrame(date_array, columns = date_labels)
date = date.astype(int)
date = date.rename(columns={1:'January', 2:'February', 3:'March', 4:'April', 5:'May', 6:'June',
                            7:'July', 8:'August', 9:'September', 10:'October', 11:'November', 12:'December'})
calendar = calendar.join(date)
calendar = calendar.drop(columns=['weekday', 'wday', 'month'])

## Merge sell_prices and calendar



*   Merge sell prices with sales_data_intermittend['item_id'] (df with 25 chosen products)
*   Keep only rows with store_id CA_3
*   Merge sell_prices & calendar --> create new df sell_prices_calendar
*   Edit sell_prices_calendar (drop some columns and sort values)
*   Delete all rows in sell_prices_calendar containing day, that are not contained in sell_data







In [ ]:
sell_prices = sell_prices.merge(sales_data_intermittend['item_id'], how='inner', on='item_id')
sell_prices = sell_prices[sell_prices['store_id'].str.contains('CA_3') == True]
sell_prices_calendar = sell_prices.merge(calendar, how='inner', on='wm_yr_wk')
sell_prices_calendar = sell_prices_calendar.sort_values(by=['date', 'item_id'])
sell_prices_calendar = sell_prices_calendar[sell_prices_calendar['d'].isin(day_cols)]

## Merge sales_price_calendar and sell_data to the final frame

*   Set the index of both dfs to item_id
*   Use the lookup function to create the demand column for each product on each day



In [ ]:
sell_prices_calendar= sell_prices_calendar.set_index('item_id')
sales_data = sales_data.set_index('item_id')
sell_prices_calendar['demand'] = sales_data.lookup(sell_prices_calendar.index, sell_prices_calendar.d)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  This is separate from the ipykernel package so we can avoid doing imports until


## Create final_set

In [ ]:
final_set = sell_prices_calendar.reset_index()
final_set = final_set.drop(columns=['d', 'wm_yr_wk', 'store_id'])
final_set = final_set.set_index('date')

In [ ]:
final_set

,item_id,sell_price,year,is_snap_event,is_sporting_event,is_national_event,is_cultural_event,is_religious_event,Friday,Monday,...,April,May,June,July,August,September,October,November,December,demand
date,,,,,,,,,,,,,,,,,,,,,
2011-01-29,FOODS_1_018,1.00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2011-01-29,FOODS_2_013,2.18,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
2011-01-29,FOODS_2_181,2.08,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,18
2011-01-29,FOODS_2_244,5.48,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,32
2011-01-29,FOODS_2_347,2.18,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-05-22,FOODS_3_694,1.68,5,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,19
2016-05-22,FOODS_3_697,3.00,5,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,2
2016-05-22,FOODS_3_702,3.00,5,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,28


*   create final_set_feature
*   create final_set_demand



In [ ]:
final_set_Y = final_set[['demand']]
final_set_X = final_set.drop('demand', axis=1)

In [ ]:
final_set.to_csv('/content/drive/MyDrive/M5/final_set.csv')
final_set_Y.to_csv('/content/drive/MyDrive/M5/final_set_Y.csv')
final_set_X.to_csv('/content/drive/MyDrive/M5/final_set_X.csv')